# setup

## data

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")
GDRIVE_PATH = "/content/gdrive/MyDrive/austral/tesis"
GDRIVE_DATA_PATH = f"{GDRIVE_PATH}/data"
GDRIVE_TRAINED_MODELS_PATH = f"{GDRIVE_PATH}/trained_models"

!cp $GDRIVE_DATA_PATH/processed_data.tar.gz processed_data.tar.gz
!tar -xf ./processed_data.tar.gz

## Modules

In [ ]:
# add path to modules to be able to import stuff
import sys
from pathlib import Path

HOME = Path.cwd()
REPO_PATH = HOME / "rtccnn"
CONFIG_PATH = REPO_PATH / "config" / "colab.yaml"

%git clone https://github.com/gerbeldo/rtccnn.git $REPO_PATH

# Get the absolute path to the 'utils' directory
utils_path = REPO_PATH / "utils"
models_path = REPO_PATH / "models"

# Add this path to sys.path
if utils_path not in sys.path:
    sys.path.append(str(utils_path.resolve()))

if models_path not in sys.path:
    sys.path.append(str(models_path.resolve()))

# Import modules

In [ ]:
from load_config import load_config
from dataset import CellDivisionDataset
from train import train_model
from simple_3d_cnn import Simple3DCNN

# Train model

In [ ]:
config = load_config(str(CONFIG_PATH))

dataset = CellDivisionDataset(
    config["data"]["annotations_path"],
    config["data"]["img_path"],
    transform=None,
    device=config["env"]["device"],
)

model = Simple3DCNN()
model = train_model(config, dataset, model)

# Save model

In [ ]:
# save model
# prepend date and an ID
from datetime import datetime

model_id = datetime.now().strftime("%Y%m%d-%H%M%S")

model_path = Path("trained_models") / f"{model_id}_model.pt"
torch.save(model, model_path)

# upload to drive
!cp {str(model_path)} {GDRIVE_TRAINED_MODELS_PATH}/{model_id}